Importar librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
#paquete para manejar varable de entorno(variable q esta en otra parte del codigo)
from dotenv import load_dotenv
import os

In [2]:
#Desactivar warnings
import warnings
warnings.filterwarnings('ignore')

Conexion a Base de Datos

In [ ]:
import os

def cargar_variables_entorno(archivo):
    # Abrir el archivo de texto con las variables
    with open(archivo, 'r') as file:
        # Leer cada línea y configurar las variables de entorno
        for line in file:
            # Limpiar posibles saltos de línea y separar la clave y valor
            clave, valor = line.strip().split('=')
            os.environ[clave] = valor  # Establecer la variable de entorno

# Llamar a la función con el archivo config.txt
cargar_variables_entorno('C:\\Users\\Public\\PF\\config.txt')

# Ahora puedes acceder a las variables de entorno
host = os.getenv('HOST')
dbName = os.getenv('DB_NAME')
port = int(os.getenv('PORT'))  # Asegúrate de convertir a entero si es necesario
user = os.getenv('USER')
pswd = os.getenv('PASSWORD')

print(f"Host: {host}, DB Name: {dbName}, Port: {port}, User: {user}, Password: {pswd}")

In [4]:
user = os.getenv('USER')
pswd = os.getenv('PASSWORD')
host = os.getenv('HOST')
dbName = os.getenv('DB_NAME')
port = int(os.getenv('PORT'))

try:
    connection = pymysql.connect(host = host, user = user, passwd = pswd, database = dbName, port = port)
    cursor = connection.cursor()
    print (cursor) # Muestra el cursor si todo funciona correctamente

except pymysql.Error as e:
    print("Error en la conexión:", e)

Creacion de los Dataframe

In [5]:
#Tabla aircrafts
query = "SELECT * FROM aircrafts_data;"
aircrafts_data = pd.read_sql_query(query, connection)

#Tabla airports
query = "SELECT * FROM airports_data;"
airports_data = pd.read_sql_query(query, connection)

#Tabla boarding_passes
query = "SELECT * FROM boarding_passes;"
boarding_passes = pd.read_sql_query(query, connection)

#Tabla bookings
query = "SELECT * FROM bookings;"
bookings = pd.read_sql_query(query, connection)

#Tabla flights
query = "SELECT * FROM flights;"
flights = pd.read_sql_query(query, connection)

#Tabla seats
query = "SELECT * FROM seats;"
seats = pd.read_sql_query(query, connection)

#Tabla ticket_flights
query = "SELECT * FROM ticket_flights;"
ticket_flights = pd.read_sql_query(query, connection)

#Tabla tickets
query = "SELECT * FROM tickets;"
tickets = pd.read_sql_query(query, connection)

Mapa de Rutas

In [6]:
# Extraer latitud y longitud usando una expresión regular
airports_data[['lat', 'lon']] = airports_data['coordinates'].str.extract(r'\(([^,]+),\s*([^)]+)\)')

# Convertir las columnas de texto a tipo float
airports_data['lat'] = airports_data['lat'].astype(float)
airports_data['lon'] = airports_data['lon'].astype(float)

In [12]:
# Unir las tablas usando 'merge' similar a un JOIN
merged_df = pd.merge(ticket_flights, flights, on='flight_id')

# Contar los tickets vendidos por flight_no
top_10_vuelos = (
    merged_df.groupby('flight_no')
    .agg(total_tickets_sold=('ticket_no', 'count'))
    .reset_index()
    .sort_values(by='total_tickets_sold', ascending=False)
    .head(5)
)

print(top_10_vuelos)

    flight_no  total_tickets_sold
150    PG0222               15903
153    PG0225               15879
154    PG0226               15433
151    PG0223               15402
152    PG0224               15155


In [13]:
flights_unique = flights.drop_duplicates(subset='flight_no')

In [15]:
# Unir con los aeropuertos de salida (departure_airport)
merged_top_5 = pd.merge(top_10_vuelos, flights_unique[['flight_no', 'departure_airport']], on='flight_no', how='left')

# Unir con los aeropuertos de llegada (arrival_airport)
merged_top_5 = pd.merge(merged_top_5, flights_unique[['flight_no', 'arrival_airport']], on='flight_no', how='left')

# Unir con las coordenadas de los aeropuertos de salida (departure_airport)
merged_top_5 = pd.merge(merged_top_5, airports_data[['airport_code', 'lat', 'lon']], left_on='departure_airport', right_on='airport_code', how='left', suffixes=('_dep', '_arr'))

# Unir con las coordenadas de los aeropuertos de llegada (arrival_airport)
merged_top_5 = pd.merge(merged_top_5, airports_data[['airport_code', 'lat', 'lon']], left_on='arrival_airport', right_on='airport_code', how='left', suffixes=('_dep', '_arr'))

merged_top_5

,flight_no,total_tickets_sold,departure_airport,arrival_airport,airport_code_dep,lat_dep,lon_dep,airport_code_arr,lat_arr,lon_arr
0,PG0222,15903,DME,OVB,DME,37.906300,55.408798,OVB,82.650703,55.012600
1,PG0225,15879,SVO,SVX,SVO,37.414600,55.972599,SVX,60.802700,56.743099
2,PG0226,15433,SVX,SVO,SVX,60.802700,56.743099,SVO,37.414600,55.972599
3,PG0223,15402,OVB,DME,OVB,82.650703,55.012600,DME,37.906300,55.408798
4,PG0224,15155,SVO,AER,SVO,37.414600,55.972599,AER,39.956600,43.449902


In [16]:
import folium
from IPython.display import display

# Crear un mapa centrado en Rusia
mapa = folium.Map(location=[61.5240, 105.3188], zoom_start=3)

# Iterar sobre el DataFrame merged_top_10 para crear líneas de rutas
for idx, row in merged_top_5.iterrows():
    # Obtener las coordenadas de los aeropuertos de salida y llegada
    lat_dep = row['lat_dep']
    lon_dep = row['lon_dep']
    lat_arr = row['lat_arr']
    lon_arr = row['lon_arr']
    
    # Crear una línea entre los puntos de salida y llegada
    folium.PolyLine(
        locations=[(lat_dep, lon_dep), (lat_arr, lon_arr)],
        color='lightblue', 
        weight=2.5,   # Grosor de la línea
        opacity=1.0   # Opacidad de la línea
    ).add_to(mapa)

    # Agregar marcadores para los aeropuertos de salida y llegada
    folium.Marker([lat_dep, lon_dep], popup=f"Departure: {row['departure_airport']}").add_to(mapa)
    folium.Marker([lat_arr, lon_arr], popup=f"Arrival: {row['arrival_airport']}").add_to(mapa)

# Mostrar el mapa directamente en el cuaderno de Jupyter
display(mapa)

In [18]:
cursor.close()
connection.close()